<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/IonQ_%EC%9D%B4%EB%A1%A0%ED%8A%B8%EB%9E%A9%EC%9E%90%EC%84%B8%ED%9E%88%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0_QFT_Adder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m pip install --upgrade qiskit
!python3 -m pip install qiskit_ionq
!python3 -m pip install matplotlib

In [ ]:
#import Aer here, before calling qiskit_ionq_provider
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
from qiskit.circuit.library import QFT

from qiskit_ionq import IonQProvider
from math import pi
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#Call provider and set token value
provider = IonQProvider(token="OiyYG9lEiTNGHJMqBF8GYAn4AIwZX4QV")
provider.backends()

In [ ]:
# Quantum Fourier transform of |q>, of Length n.
def qft(circ, q, n):
  #loop through the target qubits.
  for i in range(n, 0, -1):
    # Apply the H gate to the target.
    circ.h(q[i-1])

    # Loop through the control qubits.
    for j in range(i--1, 0, -1):
      circ.cp(2*np.pi/2**(i-j+1, q[j-1]), q[i-1])
    ###for
  ###for
###def

# Inverse Fourier transform of |q>, of Length n.
def iqft(circ, q, n):
  #Loop through the target qubits.
  for i in range(1, n+1):
    #  Loop through the control qubits.
    for j in range(1, i):
      # The inverse Fourier transform just users a negative phase.
      circ.cp(-2*np.pi/2**(i-j+1), q[j-1], q[i-1])
    ###for
    # Apply the H gate to the target.
    circ.h(q[i-1])
  ###for
###def

In [ ]:
# define the Add Function
def Add(circ, a, b, n):
  # add  1 to n to account for overflow
  n += 1
  # take the QFT
  qft(circ, b, n)

  circ.barrier()

  # Compute the controlled phases
  # Iterate over targets
  for i in range(n, 0, -1):
    # Interate over controls
    for j in range(i, 0, -1):
      # If the qubit a[j-1] exists run cp, if not assume the qubit is 0 and naver existed.
      if len(a) - 1 >= j - 1:
        circ.cp(2*np.pi/2**(i-j+1), a[j-1], b[i-1])
      ###if
    ###for
  ###for
  circ.barrier()

  # take the inverse QFT
  iqft(circ, b, n)
###def

In [ ]:
# Registers and circuit.
a = QuantumRegister(2)
b = QuantumRegister(3)
ca = ClassicalRegister(2)
cb = ClassicalRegister(3)
qc = QuantumCircuit(a, b, ca, cb)

# Numbers to add.
qc.x(a[1]) # a =  01110 / a = 10
#qc.x(a[2])
#qc.x(a[3])

qc.x(b[0]) # b = 01011 / b = 001
#qc.x(b[1])
#qc.x(b[3])

qc.barrier()

# Add the numbers, so |a> |b> to |a>|a+b>.
Add(qc, a, b, 2)

qc.barrier()

# Measure the results
qc.measure(a, ca)
qc.measure(b, cb)

qc.draw(output='mpl')

In [ ]:
# Simulate the circuit
simulator = Aer.get_backend('qasm_simulator')
job_sim = execute(qc, simulator)
result_sim = job_sim.result()

print(result_sim.get_counts(qc))


In [ ]:
# run the circuit on a real-device
qpu = provider.get_backend('ionq_qpu')
qpu_job = qpu.run(qc)

# Check if job is done.
while qpu_job.status() is not JobStatus.DONE:
  print("Job status is", qpu_job.status())
  time.sleep(60)
###while

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(qpu_job.get_counts())